## <center> $$\text{Trabalho II}$$
- Dada a gramática LR(1) da Figura 1. Você deve Implementar:
    - (3,5 pontos) Um algoritmo que calcula os conjuntos FISRT e FOLLOW.
    - (3,5 pontos) Um algoritmo que constrói o Autômato LR(1).
    - (3,0 pontos) Um algoritmo para o reconhecimento sintático. Isto é, dada uma palavra w, o seu analisador deve ser capaz de dizer se w obedece ou não as regras da gramática.

In [9]:
from typing import List
import copy

In [2]:
A =[1,2,3, 4] 
B = [4]

In [3]:
list(set(A) -set(B))

[1, 2, 3]

In [5]:
a = set([1,2,3])

In [72]:
class Language:
    """
    Classe para representar uma linguagem
    """
    def __init__(self, terminals, no_terminals, void_symbol='#', **rules: dict):
        self.productions = {}
        for word, rules in rules.items():
            self.productions[word] = rules
            setattr(self, word, rules)
        
        self.terminals = terminals
        self.no_terminals = no_terminals
        self.void = void_symbol 

        self._first, self._follow = {}, {}


    def __repr__(self) -> str:
        return self.productions.__str__()


    def _remove_left_single_recursion(self, word, inplace=False):
        new_lang = copy.deepcopy(self)
    
        WORD_LINE = f'{word}\''

        productions = getattr(new_lang, word)
        new_word_productions = []
        word_line_productions = []
        for prod in productions:
            if prod[0] == word:
                word_line_productions.append(prod[1:]+[WORD_LINE])
            else:
                new_word_productions.append(prod+[WORD_LINE])
        word_line_productions.append([self.void])

        new_lang.productions[word] = new_word_productions
        new_lang.productions[WORD_LINE] = word_line_productions

        setattr(new_lang, word, new_word_productions)
        setattr(new_lang, WORD_LINE, word_line_productions)

        if not inplace:
            return new_lang
        
        self.productions = new_lang.productions
        for word, productions in new_lang.productions.items():
            setattr(self, word, productions)
    

    def remove_left_recursion(self, word = None, inplace=False):
        if word is not None:
            return self._remove_left_single_recursion(word, inplace)
        
        new_lang = copy.deepcopy(self)

        for word_ in new_lang.productions.keys():
            if self.isrecursive(word_):
                new_lang._remove_left_single_recursion(word_, True)
        
        if not inplace:
            return new_lang
        
        self.productions = new_lang.productions
        for word, productions in new_lang.productions.items():
            setattr(self, word, productions)
        return self
    

    def _check_recursion(self, word, first_word):
        if first_word not in self.productions:
            return False
        if first_word == word:
            return True
        for prod in self.productions[first_word]:
            if prod[0] == first_word:
                return False
            if prod[0] in self.productions:
                return self._check_recursion(word, prod[0])
        return False


    def isrecursive(self, word: str) -> bool:
        return True in [
            self._check_recursion(word, prod[0])
            for prod in self.productions[word]]


    def first(self, X):
        first_ = []
        if X in self.terminals:
            return [X]
        for production in self.productions[X]:
            if production[0] not in self.productions:
                first_.append(production[0])
            else:
                first_ += self.first(production[0])
        return first_
            

In [73]:
lang = Language(
    terminals=['+','*', 'id', '(', ')'],
    no_terminals=['E','T', 'F'],
    E = [['E','+','T'],['T']],
	T = [['T','*','F'],['F']],
	F = [['(', 'E', ')'], ['id']]
)

In [74]:
lang.productions

{'E': [['E', '+', 'T'], ['T']],
 'T': [['T', '*', 'F'], ['F']],
 'F': [['(', 'E', ')'], ['id']]}

In [75]:
lang.remove_left_recursion(inplace=True)

{'E': [['T', "E'"]], 'T': [['F', "T'"]], 'F': [['(', 'E', ')'], ['id']], "E'": [['+', 'T', "E'"], ['#']], "T'": [['*', 'F', "T'"], ['#']]}

In [76]:
lang.productions

{'E': [['T', "E'"]],
 'T': [['F', "T'"]],
 'F': [['(', 'E', ')'], ['id']],
 "E'": [['+', 'T', "E'"], ['#']],
 "T'": [['*', 'F', "T'"], ['#']]}

In [77]:
lang.first('E')

['(', 'id']

In [78]:
lang = Language(
    terminals=['+',' ', '(', ')', '*', 'i'],
    no_terminals=['E','T', 'F'],
    E = [['T','R']],
	R = [['+','T', 'R'],['#']],
	T = [['F', 'Y']],
    Y = [['*', 'F', 'Y'], ['#']],
    F = [['(', 'E', ')'], ['i']]
)

In [79]:
lang.productions

{'E': [['T', 'R']],
 'R': [['+', 'T', 'R'], ['#']],
 'T': [['F', 'Y']],
 'Y': [['*', 'F', 'Y'], ['#']],
 'F': [['(', 'E', ')'], ['i']]}

In [80]:
lang.first('R')

['+', '#']